In [1]:
import numpy as np
# https://medium.com/@stevehiehn/how-to-generate-music-with-python-the-basics-62e8ea9b99a5
from midiutil import MIDIFile
from mingus.core import chords
from IPython.display import Audio
import os

In [2]:
NOTES = ['C', 'C#', 'D', 'Eb', 'E', 'F', 'F#', 'G', 'Ab', 'A', 'Bb', 'B']
BASERANGE = range(35, 55)
BASERANGE2 = range(55, 75)
MAJORMINOR = ['M', 'm']
SEVENTH = [''] # TODO add 7


def note_to_number(note):
    def swap_accidentals(note):
        if note == 'Cb':
            return 'B'
        if note == 'Db':
            return 'C#'
        if note == 'D#':
            return 'Eb'
        if note == 'E#':
            return 'F'
        if note == 'Gb':
            return 'F#'
        if note == 'G#':
            return 'Ab'
        if note == 'A#':
            return 'Bb'
        if note == 'B#':
            return 'C'

        return note
    note = swap_accidentals(note)
    return NOTES.index(note)

track = 0
channel = 0
time = 0  # In beats
duration = 3  # In beats
tempo = 60  # In BPM
volume = 100  # 0-127, as per the MIDI standard
n_repeats = 10

In [3]:
def generate_midi():
    base = np.random.choice(NOTES)
    majorminor = np.random.choice(MAJORMINOR)
    seventh = np.random.choice(SEVENTH)
    chord = base + majorminor + seventh

    MyMIDI = MIDIFile(1)  # One track, defaults to format 1 (tempo track is created
    # automatically)
    MyMIDI.addTempo(track, time, tempo)

    notes = chords.from_shorthand(chord)
    # print(chord, notenums)
    for i in range(n_repeats):
        notenums = []
        for baserange in [BASERANGE, BASERANGE2]:
            base = np.random.choice(baserange)
            baseC = (base // 12) * 12
            basenote = base % 12
            for note in notes:
                notenum = note_to_number(note)
                if notenum < basenote:
                    notenums.append(baseC + notenum + 12)
                else:
                    notenums.append(baseC + notenum)
        for notenum in notenums:
            MyMIDI.addNote(track, channel,
                           notenum, i * duration,
                           duration, volume)
    with open('out.mid', 'wb') as output_file:
        MyMIDI.writeFile(output_file)
    os.system('timidity out.mid')
    return chord


In [8]:
chord = generate_midi()
Audio('out.ogg', autoplay=True)

Playing out.mid
MIDI file: out.mid
Format: 1  Tracks: 2  Divisions: 960
Output out.ogg
Playing time: ~34 seconds
Notes cut: 0
Notes lost totally: 0


In [9]:
guess = input('Guess: ')
n_guess = 1
while guess != chord:
    if guess == 'break':
        print('Gave up, it was', chord)
        break
    print('WRONG')
    n_guess += 1

    guess = input('Guess: ')
else:
    print('Yes, it was %s, took %d tries' % (chord, n_guess))

Guess: BbM
Yes, it was BbM, took 1 tries
